## 4. Постановка эксперимента

In [6]:
import numpy as np 
import pandas as pd
from sklearn import preprocessing 
import pickle
import warnings
warnings.filterwarnings("ignore")
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing 
from sklearn.model_selection import train_test_split

Подгрузим обученную модель RandomForestRegressor

In [7]:
# Загрузим сохраненную модель
with open('model/working_model.pkl', "rb") as m:
    work_model = pickle.load(m)

In [8]:
# создадим тестовый фрейм данных
data = [
    (1, 1, 'Round Rock', 324, 78680, 'TX', 0, 4.0, 1.0, 1995, 0, 0, 1, 1, False, False, True, False, False, False, False, False, False, False),
    (2, 1, 'Los Angeles', 1545, 90030, 'CA',  1, 8.0, 2.5, 2015, 0, 1, 0, 1, False, False, True, False, False, False, False, False, False, False),
    (1, 0, 'Round Rock',  674, 78680, 'TX',  1, 6.0, 1.5, 1998, 0, 1, 0, 1, False, False, True, False, False, False, False, False, False, False),
    (3, 1, 'Los Angeles', 1098, 90030, 'CA', 1, 7.0, 2.0, 2010, 0, 1, 1, 1, False, False, True, False, False, False, False, False, False, False),
    (1, 0, 'Round Rock', 386, 78680, 'TX', 0, 5.0, 0.5, 2005, 0, 0, 0, 0, False, False, True, False, False, False, False, False, False, False)
]
columns = [
    'baths', 'fireplace', 'city', 'sqft', 'zipcode', 'state', 'pool', 'school_max_rating', 'shcool_mean_distance', 
    'Year built', 'Remodeled', 'Heating', 'Cooling', 'Parking', 'propertyType_apartment', 'propertyType_condo',
    'propertyType_family_home', 'propertyType_historical', 'propertyType_land', 'propertyType_mobile_home',
    'propertyType_modern', 'propertyType_other', 'propertyType_ranch', 'propertyType_townhouse'
]
df_test = pd.DataFrame(data, columns=columns)
df_test.head()

,baths,fireplace,city,sqft,zipcode,state,pool,school_max_rating,shcool_mean_distance,Year built,...,propertyType_apartment,propertyType_condo,propertyType_family_home,propertyType_historical,propertyType_land,propertyType_mobile_home,propertyType_modern,propertyType_other,propertyType_ranch,propertyType_townhouse
0,1,1,Round Rock,324,78680,TX,0,4.0,1.0,1995,...,False,False,True,False,False,False,False,False,False,False
1,2,1,Los Angeles,1545,90030,CA,1,8.0,2.5,2015,...,False,False,True,False,False,False,False,False,False,False
2,1,0,Round Rock,674,78680,TX,1,6.0,1.5,1998,...,False,False,True,False,False,False,False,False,False,False
3,3,1,Los Angeles,1098,90030,CA,1,7.0,2.0,2010,...,False,False,True,False,False,False,False,False,False,False
4,1,0,Round Rock,386,78680,TX,0,5.0,0.5,2005,...,False,False,True,False,False,False,False,False,False,False


Проведем кодирование признаков в тестовом наборе данных

In [9]:
r_scaler = preprocessing.RobustScaler()
for column in ['baths', 'sqft', 'school_max_rating', 'shcool_mean_distance']:
    df_test[column] = r_scaler.fit_transform(df_test[[column]])[:,0]

In [10]:
df_test['zipcode'] = df_test['zipcode'].astype(str)
df_test['Year built'] = df_test['Year built'].astype(str)

In [11]:
le = LabelEncoder()
df_test['state'] = le.fit_transform(df_test['state'])
df_test['city'] = le.fit_transform(df_test['city'])
df_test['zipcode'] = le.fit_transform(df_test['zipcode'])
df_test['Year built'] = le.fit_transform(df_test['Year built'])

In [12]:
df_test

,baths,fireplace,city,sqft,zipcode,state,pool,school_max_rating,shcool_mean_distance,Year built,...,propertyType_apartment,propertyType_condo,propertyType_family_home,propertyType_historical,propertyType_land,propertyType_mobile_home,propertyType_modern,propertyType_other,propertyType_ranch,propertyType_townhouse
0,0.0,1,1,-0.491573,0,1,0,-1.0,-0.5,0,...,False,False,True,False,False,False,False,False,False,False
1,1.0,1,0,1.223315,1,0,1,1.0,1.0,4,...,False,False,True,False,False,False,False,False,False,False
2,0.0,0,1,0.000000,0,1,1,0.0,0.0,1,...,False,False,True,False,False,False,False,False,False,False
3,2.0,1,0,0.595506,1,0,1,0.5,0.5,3,...,False,False,True,False,False,False,False,False,False,False
4,0.0,0,1,-0.404494,0,1,0,-0.5,-1.0,2,...,False,False,True,False,False,False,False,False,False,False


Произведем предсказание стоимости объектов недвижимости

In [13]:
predicted_prices = work_model.predict(df_test)

# Вывод предсказанных стоимостей
print("Предсказанные стоимости объектов недвижимости:")
for price in predicted_prices:
    print(f'{round(price)} $')

Предсказанные стоимости объектов недвижимости:
254335 $
412520 $
300057 $
335275 $
212635 $


#### Модель работает

In [14]:
# ! pip freeze > requirements.txt